<a href="https://colab.research.google.com/github/cerr/CERR/blob/octave_dev/Jupyter_Notebooks/demo_radiomic_and_dosimetric_feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#  Example of radiomic & dosimetric feature calculation using CERR

### Install latest Octave from compile

In [1]:
%%capture
! apt-get update
! apt-get install libgraphicsmagick++1-dev libsuitesparse-dev libqrupdate1 libfftw3-3 gnuplot zsh
! pip install oct2py octave_kernel

In [2]:
import os, urllib.request, json

In [3]:
%%capture
# download compiled octave package (latest)
def get_CERRoctave(root_path):
  os.chdir(root_path)
  with urllib.request.urlopen("https://api.github.com/repos/cerr/octave-colab/releases/latest") as url:
      data = json.loads(url.read().decode())
  fname = data['assets'][0]['name']
  requrl = data['assets'][0]['browser_download_url']
  urllib.request.urlretrieve(requrl, fname)
  # unzip, identify octave folder name
  !tar xf {fname}
  top_folder = !tar tf {fname} | head -1
  octave_folder = top_folder[0][:-1]
  octave_path = os.path.join(root_path,octave_folder)
  !git clone https://github.com/cerr/CERR.git
  !cd CERR && git checkout octave_dev
  cerr_path = os.path.join(root_path,'CERR')
  return octave_path, cerr_path


octave_path,cerr_path = get_CERRoctave('/content')
os.environ['OCTAVE_EXECUTABLE'] = octave_path + '/bin/octave-cli'
os.environ['PATH'] = octave_path + '/bin:' + os.environ['PATH']

#  Install reqd Octave packages

In [4]:
%%capture
%load_ext oct2py.ipython

!apt install octave liboctave-dev
!wget https://nchc.dl.sourceforge.net/project/octave/Octave%20Forge%20Packages/Individual%20Package%20Releases/image-2.12.0.tar.gz
!wget https://nchc.dl.sourceforge.net/project/octave/Octave%20Forge%20Packages/Individual%20Package%20Releases/io-2.6.1.tar.gz
!wget https://nchc.dl.sourceforge.net/project/octave/Octave%20Forge%20Packages/Individual%20Package%20Releases/statistics-1.4.2.tar.gz

!octave --eval "pkg install image-2.12.0.tar.gz"
!octave --eval "pkg install io-2.6.1.tar.gz"
!octave --eval "pkg install statistics-1.4.2.tar.gz"


### Download CERR


In [5]:
%%capture
!git clone https://github.com/cerr/CERR.git
%cd /content/CERR
!git checkout octave_dev
%cd ..

In [6]:
%%capture
!git clone --single-branch --branch octave_dev https://www.github.com/cerr/CERR.git
import os
currDir = os.getcwd()
os.chdir("/content/CERR")
!git checkout 2847d7df4a889c2ebd9c93a9d25bf46fd2ee75be
os.chdir(currDir)

### Add Octave packages and CERR to path

In [7]:
%%octave
pkg load image
pkg load io
pkg load statistics

### Read radiomics calculation settings

In [8]:
%%octave
%%capture
#Add CERR to path
cerrPath = '/content/CERR'
currDir = pwd;
cd(cerrPath)
addToPath2(cerrPath)
cd(currDir)

sampleData = '/content/CERR/Unit_Testing/data_for_cerr_tests/CERR_plans/head_neck_ex1_20may03.mat.bz2';
planC = loadPlanC(sampleData, tempdir);
planC = updatePlanFields(planC);
planC = quality_assure_planC(sampleData, planC);

sampleParam = '/content/CERR/Jupyter_Notebooks/demoParams.json';
paramS = getRadiomicsParamTemplate(sampleParam);

### Run feature calculation

In [9]:
%%octave
indexS = planC{end};
strC = {planC{indexS.structures}.structureName};
strName = paramS.structuresC{1};
structNum = getMatchingIndex(strName,strC,'exact');
scanNum = getStructureAssociatedScan(structNum,planC);
doseNum = 1;
binWidth = 0.01;

structFieldName = ['struct_',repSpaceHyp(strName)];

In [10]:
%%capture
%%octave

%% Compute radiomic features
radiomicsFeatS = calcGlobalRadiomicsFeatures...
(scanNum, structNum, paramS, planC);

%% Compute DVH-based features
[doseV,volV] = getDVH(structNum,doseNum,planC);
[doseBinsV,volHistV] = doseHist(doseV,volV,binWidth);
dvhFeatS.meanDose = calc_meanDose(doseBinsV, volHistV);
dvhFeatS.d10 = calc_Dx(doseBinsV, volHistV, 10, 1);
dvhFeatS.v30 = calc_Vx(doseBinsV, volHistV, 30, 0);
dvhFeatS.MOH55 = calc_MOHx(doseBinsV, volHistV, 55);

### Make computed features available in Python 


In [11]:
from oct2py import octave

%octave_pull radiomicsFeatS dvhFeatS
print(radiomicsFeatS['shapeS'])
print(dvhFeatS)

{'majorAxis': 12.732470146787053, 'minorAxis': 8.91029738209561, 'leastAxis': 4.082517261054258, 'flatness': 0.32063827474077783, 'elongation': 0.6998090142268317, 'max3dDiameter': 10.802777290344238, 'max2dDiameterAxialPlane': 9.492101669311523, 'max2dDiameterSagittalPlane': 6.020797252655029, 'max2dDiameterCoronalPlane': 10.581587791442871, 'surfArea': 78.75934058605056, 'volume': 3.4029999999999943, 'filledVolume': 3.4029999999999943, 'Compactness1': 0.0027468434324124952, 'Compactness2': 0.0026808348319102646, 'spherDisprop': 7.198521632892385, 'sphericity': 0.13891741262965365, 'surfToVolRatio': 23.144090680590857}
{'meanDose': 46.96107504826258, 'd10': 66.865, 'v30': 32.515999999999984, 'MOH55': 55.1462897603486}
